In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

import scipy
import scipy.special as special
import scipy.stats as stats
import scipy.integrate as integrate

The error function is an entire function; it has no singularities (except that at infinity) and its Taylor expansion always converges.

The defining integral cannot be evaluated in closed form in terms of elementary functions, but by expanding the integrand $e^{−z^2}$ into its Maclaurin series and integrating term by term, one obtains the error function's Maclaurin series as:

$$ \operatorname {erf} (z)={\frac {2}{\sqrt {\pi }}}\sum _{n=0}^{\infty }{\frac {(-1)^{n}z^{2n+1}}{n!(2n+1)}}={\frac {2}{\sqrt {\pi }}}\left(z-{\frac {z^{3}}{3}}+{\frac {z^{5}}{10}}-{\frac {z^{7}}{42}}+{\frac {z^{9}}{216}}-\cdots \right) $$

which holds for every complex number z. 

### Tasks
The special.erf method is vectorized so that we can obtain multiple values for input sequences and grid

#### Write a vectorized function using the Taylor series algorithm. 

In [67]:
def erf_taylor(z, nterms=19):
    
    if type(z)==int:
        length = 1
    else:
        length = len(z)
    
    #Taylor series coefficients
    n = np.arange(0, nterms, dtype=np.double).reshape(nterms,1)
    terms = (-1)**n* z**(2*n+1) / special.factorial(n) / (2*n+1)
    
    z_array = np.tile(z,nterms).reshape(nterms,length)
    
    return np.sum(terms*z_array, axis=0) * 2.0 / np.sqrt(np.pi)

In [82]:
z=[100]
erf_taylor(z,nterms=10)

array([-1.63493975e+33])

#### Compare results to special.erf and determine relative precision of the Taylor series algorithm for $|z| < 9$.

#### Write python code for some more complex algorithm that you find on the web ie.

Efficient Computation of the Complex Error Function
Walter Gautschi
SIAM Journal on Numerical Analysis
Vol. 7, No. 1 (Mar., 1970), pp. 187-198

Computation of the Complex Error Function
J. A. C. Weideman
SIAM Journal on Numerical Analysis
Vol. 31, No. 5 (Oct., 1994), pp. 1497-1518

More efficient computation of the complex error function.
G. P. M. Poppe and C. M. J. Wijers. 1990
ACM Trans. Math. Softw. 16, 1 (March 1990), 38-46. 
DOI: https://doi.org/10.1145/77626.77629